**This notebook is an exercise in the [SQL](https://www.kaggle.com/learn/intro-to-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/dansbecker/group-by-having-count).**

---


# Introduction

Queries with **GROUP BY** can be powerful. There are many small things that can trip you up (like the order of the clauses), but it will start to feel natural once you've done it a few times. Here, you'll write queries using **GROUP BY** to answer questions from the Hacker News dataset.

Before you get started, run the following cell to set everything up:

# I used this to fix the issue 

There was a problem with this exercise, so I found this fix. 

https://www.kaggle.com/learn/intro-to-sql/discussion/417044#2316394

In [1]:
from learntools.core import binder
binder.bind(globals())
# Read ex3.py and apply some "fixes"
with open("/opt/conda/lib/python3.10/site-packages/learntools/sql/ex3.py","r") as f:
    ex3 = f.read()
    ex3 = ex3.replace("SELECT author","SELECT `by`") # need to use backtick since column name is 'by' which is also a SQL keyword
    ex3 = ex3.replace("GROUP BY author","GROUP BY `by`")
    ex3 = ex3.replace("\'author\'","\'by\'")
    ex3 = ex3.replace("`author`","\`by\`")
    ex3 = ex3.replace("bigquery-public-data.hacker_news.comments","bigquery-public-data.hacker_news.full")

    # Write fixes into v2 file
    with open("/opt/conda/lib/python3.10/site-packages/learntools/sql/ex3_v2.py","w") as f2:
        f2.write(ex3)

# Verify file looks good
# with open("/opt/conda/lib/python3.10/site-packages/learntools/sql/ex3_v2.py","r") as f:
#     print(f.read())

# Import v2
from learntools.sql.ex3_v2 import *

Using Kaggle's public dataset BigQuery integration.


The code cell below fetches the `comments` table from the `hacker_news` dataset.  We also preview the first five rows of the table.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "comments" table
table_ref = dataset_ref.table("full") 

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "comments" table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,"I would rather just have wired earbuds, period...",None,zeveb,NaN,1591717736,2020-06-09 15:48:56+00:00,comment,23467666,23456782,NaN,NaN,None
1,None,None,DNS?,None,nly,NaN,1572810465,2019-11-03 19:47:45+00:00,comment,21436112,21435130,NaN,NaN,None
2,None,None,These benchmarks seem pretty good. Filterable...,None,mrkeen,NaN,1591717727,2020-06-09 15:48:47+00:00,comment,23467665,23467426,NaN,NaN,None
3,None,None,Oh really?<p>* Excel alone uses 86.1MB of priv...,None,oceanswave,NaN,1462987532,2016-05-11 17:25:32+00:00,comment,11677248,11676886,NaN,NaN,None
4,None,None,These systems are useless. Of the many flaws:...,None,nyxxie,NaN,1572810473,2019-11-03 19:47:53+00:00,comment,21436113,21435025,NaN,NaN,None


# Exercises

### 1) Prolific commenters

Hacker News would like to send awards to everyone who has written more than 10,000 posts. Write a query that returns all authors with more than 10,000 posts as well as their post counts. Call the column with post counts `NumPosts`.

In case sample query is helpful, here is a query you saw in the tutorial to answer a similar question:
```
query = """
        SELECT parent, COUNT(1) AS NumPosts
        FROM `bigquery-public-data.hacker_news.comments`
        GROUP BY parent
        HAVING COUNT(1) > 10
        """
```

In [3]:
# Query to select prolific commenters and post counts
prolific_commenters_query = """
                            SELECT `by`, COUNT(1) AS NumPosts
                            FROM `bigquery-public-data.hacker_news.full`
                            GROUP BY `by`
                            HAVING COUNT(1) > 10000
                            """ 

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(prolific_commenters_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
prolific_commenters = query_job.to_dataframe()

# View top few rows of results
print(prolific_commenters.head())

# Check your answer
q_1.check()

             by  NumPosts
0       rhizome     11014
1         gruez     11242
2   chrisseaton     13650
3      ceejayoz     17201
4  dragonwriter     46333


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [4]:
# q_1.solution()

### 2) Deleted comments

How many comments have been deleted? (If a comment was deleted, the `deleted` column in the comments table will have the value `True`.)

In [5]:
# Write your query here and figure out the answer

# Query to select prolific commenters and post counts
deleted_comments = """
                    SELECT COUNT(deleted) AS DeletedComments
                    FROM `bigquery-public-data.hacker_news.full`
                    WHERE deleted = True
                    """ 

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(deleted_comments, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
all_deleted_comments = query_job.to_dataframe()

# View top few rows of results
print(all_deleted_comments.head())


   DeletedComments
0           968172


In [6]:
num_deleted_posts = 968172

# Check your answer
q_2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [7]:
#q_2.solution()

# Keep Going
**[Click here](https://www.kaggle.com/dansbecker/order-by)** to move on and learn about the **ORDER BY** clause.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intro-to-sql/discussion) to chat with other learners.*